# 数据规整：连接、联合与重塑 Data Wrangling: Join, Combine, 

In [1]:
import numpy as np
import pandas as pd
pd.options.display.max_rows = 20
np.random.seed(12345)
import matplotlib.pyplot as plt
plt.rc('figure', figsize=(10, 6))
np.set_printoptions(precision=4, suppress=True)

## 8.1 分层索引 Hierarchical Indexing

直接看列子

九个数，三个a，两个b，两个c，两个d

a里面有三个，编号为123

b里面有两个，编号为13

好吧

In [2]:
data = pd.Series(np.random.randn(9),
                 index=[['a', 'a', 'a', 'b', 'b', 'c', 'c', 'd', 'd'],
                        [1, 2, 3, 1, 3, 1, 2, 2, 3]])
data

a  1   -0.204708
   2    0.478943
   3   -0.519439
b  1   -0.555730
   3    1.965781
c  1    1.393406
   2    0.092908
d  2    0.281746
   3    0.769023
dtype: float64

查看数据的索引，返回MultiIndex

里面使用索引来标注一行的索引是什么

In [3]:
data.index

MultiIndex(levels=[['a', 'b', 'c', 'd'], [1, 2, 3]],
           codes=[[0, 0, 0, 1, 1, 2, 2, 3, 3], [0, 1, 2, 0, 2, 0, 1, 1, 2]])

查看

In [4]:
data['b']

1   -0.555730
3    1.965781
dtype: float64

In [5]:
data['b':'c']

b  1   -0.555730
   3    1.965781
c  1    1.393406
   2    0.092908
dtype: float64

In [6]:
data.loc[['b', 'd']]

b  1   -0.555730
   3    1.965781
d  2    0.281746
   3    0.769023
dtype: float64

所有的2

In [7]:
data.loc[:, 2]

a    0.478943
c    0.092908
d    0.281746
dtype: float64

变成dataframe

In [11]:
data.unstack(fill_value = 0)

,1,2,3
a,-0.204708,0.478943,-0.519439
b,-0.555730,0.000000,1.965781
c,1.393406,0.092908,0.000000
d,0.000000,0.281746,0.769023


反操作

In [12]:
data.unstack().stack()

a  1   -0.204708
   2    0.478943
   3   -0.519439
b  1   -0.555730
   3    1.965781
c  1    1.393406
   2    0.092908
d  2    0.281746
   3    0.769023
dtype: float64

在DataFrame中，每个轴都可以有分层索引

In [13]:
frame = pd.DataFrame(np.arange(12).reshape((4, 3)),
                     index=[['a', 'a', 'b', 'b'], [1, 2, 1, 2]],
                     columns=[['Ohio', 'Ohio', 'Colorado'],
                              ['Green', 'Red', 'Green']])
frame

Ohio     Colorado
    Green Red    Green
a 1     0   1        2
  2     3   4        5
b 1     6   7        8
  2     9  10       11

横轴纵轴两层索引分别加入表头

In [14]:
frame.index.names = ['key1', 'key2']
frame.columns.names = ['state', 'color']
frame

state      Ohio     Colorado
color     Green Red    Green
key1 key2                   
a    1        0   1        2
     2        3   4        5
b    1        6   7        8
     2        9  10       11

查看

In [15]:
frame['Ohio']

color      Green  Red
key1 key2            
a    1         0    1
     2         3    4
b    1         6    7
     2         9   10

### 8.1.1 重排序与层级排序 Reordering and Sorting Levels

swaplevel接受两个层级序号或层级名称，返回一个进行了层级变更的对象

说人话，也就是交叉分类，分类次序调整

以前是先key1，后key2

现在变成先key2，后key1

In [16]:
frame.swaplevel('key1', 'key2')

state      Ohio     Colorado
color     Green Red    Green
key2 key1                   
1    a        0   1        2
2    a        3   4        5
1    b        6   7        8
2    b        9  10       11

sort_index，按哪一层排序

In [18]:
frame.sort_index(level=0) # 按key1排序
# frame.sort_index(level=1) # 按key2排序

state      Ohio     Colorado
color     Green Red    Green
key1 key2                   
a    1        0   1        2
     2        3   4        5
b    1        6   7        8
     2        9  10       11

交换分类层级，且按交换后的1层排序

In [19]:
frame.swaplevel(0, 1).sort_index(level=1)

state      Ohio     Colorado
color     Green Red    Green
key2 key1                   
1    a        0   1        2
2    a        3   4        5
1    b        6   7        8
2    b        9  10       11

### 8.1.2 按层级进行汇总统计 Summary Statistics by Level

在特定轴上进行聚合

In [20]:
frame.sum(level='key2')

state  Ohio     Colorado
color Green Red    Green
key2                    
1         6   8       10
2        12  14       16

可以选择横轴留下什么，我们横轴有两个分类，一个是state，一个是color

In [21]:
frame.sum(level='color', axis=1)

color      Green  Red
key1 key2            
a    1         2    1
     2         8    4
b    1        14    7
     2        20   10

In [22]:
frame.sum(level='state', axis=1)

state      Ohio  Colorado
key1 key2                
a    1        1         2
     2        7         5
b    1       13         8
     2       19        11

### 8.1.3 使用DataFrame的列进行索引 Indexing with a DataFrame's columns

看一下这个例子

In [23]:
frame = pd.DataFrame({'a': range(7), 'b': range(7, 0, -1),
                      'c': ['one', 'one', 'one', 'two', 'two',
                            'two', 'two'],
                      'd': [0, 1, 2, 0, 1, 2, 3]})
frame

,a,b,c,d
0,0,7,one,0
1,1,6,one,1
2,2,5,one,2
3,3,4,two,0
4,4,3,two,1
5,5,2,two,2
6,6,1,two,3


将c、d两列作为索引

set_index会生成一个新的DataFrame

In [24]:
frame2 = frame.set_index(['c', 'd'])
frame2

a  b
c   d      
one 0  0  7
    1  1  6
    2  2  5
two 0  3  4
    1  4  3
    2  5  2
    3  6  1

默认情况先，列会被移动到索引位置

如果既想作为索引，又要保留原始列，可以选择drop=False

In [25]:
frame.set_index(['c', 'd'], drop=False)

a  b    c  d
c   d              
one 0  0  7  one  0
    1  1  6  one  1
    2  2  5  one  2
two 0  3  4  two  0
    1  4  3  two  1
    2  5  2  two  2
    3  6  1  two  3

reset_index()是set_index()的反操作

In [26]:
frame2.reset_index()

,c,d,a,b
0,one,0,0,7
1,one,1,1,6
2,one,2,2,5
3,two,0,3,4
4,two,1,4,3
5,two,2,5,2
6,two,3,6,1


## 8.2 联合与合并数据集 Combining and Merging Datasets

各种合并方式

In [1]:
import numpy as np
import pandas as pd
pd.options.display.max_rows = 20
np.random.seed(12345)
import matplotlib.pyplot as plt
plt.rc('figure', figsize=(10, 6))
np.set_printoptions(precision=4, suppress=True)

### 8.2.1 数据库风格数据合并 Database-Style DataFrame Joins

#### merge

随便生成一个DataFrame

key:abc

data1

In [2]:
df1 = pd.DataFrame({'key': ['b', 'b', 'a', 'c', 'a', 'a', 'b'],
                    'data1': range(7)})
df1

,key,data1
0,b,0
1,b,1
2,a,2
3,c,3
4,a,4
5,a,5
6,b,6


再生成一个，为了说明merge默认的合并效果，我这里改了一下

key:abd,但是有两个b

data2

In [3]:
df2 = pd.DataFrame({'key': ['a', 'b', 'd','b'],
                    'data2': range(4)})
df2

,key,data2
0,a,0
1,b,1
2,d,2
3,b,3


#### merge情况1：没指定用什么链接么连接，默认用名字相同的列连接，也就是key

直接看效果

In [5]:
pd.merge(df1, df2)

,key,data1,data2
0,b,0,1
1,b,0,3
2,b,1,1
3,b,1,3
4,b,6,1
5,b,6,3
6,a,2,0
7,a,4,0
8,a,5,0


key值中，b和a留下，不重合的部分c和d删除了

其中b

在data1有 0,1,6三种

data2有1，3两种

所以在merge之后的表连，共有6种

a同理



#### merge情况2：指定按key这一列的值链接


In [34]:
pd.merge(df1, df2, on='key')

,key,data1,data2
0,b,0,1
1,b,1,1
2,b,6,1
3,a,2,0
4,a,4,0
5,a,5,0


#### merge情况3：左右两个表分别指定连接的列

下面看另一个例子

https://www.youtube.com/watch?v=vv4rYbqq7k0&list=PL8xPPUJdubH4rAr3gw8q8-zYRIsA-QaBl&index=27

10:37

In [35]:
df3 = pd.DataFrame({'lkey': ['b', 'b', 'a', 'c', 'a', 'a', 'b'],
                    'data1': range(7)})
df3

,lkey,data1
0,b,0
1,b,1
2,a,2
3,c,3
4,a,4
5,a,5
6,b,6


In [36]:
df4 = pd.DataFrame({'rkey': ['a', 'b', 'd'],
                    'data2': range(3)})
df4

,rkey,data2
0,a,0
1,b,1
2,d,2


In [33]:
pd.merge(df3, df4, left_on='lkey', right_on='rkey')

,lkey,data1,rkey,data2
0,b,0,b,1
1,b,1,b,1
2,b,6,b,1
3,a,2,a,0
4,a,4,a,0
5,a,5,a,0


上面默认删除c和d，如果像保留，可是设置how参数

In [37]:
pd.merge(df1, df2, how='outer')

,key,data1,data2
0,b,0.0,1.0
1,b,1.0,1.0
2,b,6.0,1.0
3,a,2.0,0.0
4,a,4.0,0.0
5,a,5.0,0.0
6,c,3.0,NaN
7,d,NaN,2.0


##### 参数 how

In [38]:
df1 = pd.DataFrame({'key': ['b', 'b', 'a', 'c', 'a', 'b'],
                    'data1': range(6)})
df1

,key,data1
0,b,0
1,b,1
2,a,2
3,c,3
4,a,4
5,b,5


In [48]:
df1 = pd.DataFrame({'key': ['b', 'b', 'a', 'c', 'a', 'b'],
                    'data1': range(6)})
df2 = pd.DataFrame({'key': ['a', 'b', 'a', 'b', 'd'],
                    'data2': range(5)})
pd.merge(df1, df2, on='key', how='left')

,key,data1,data2
0,b,0,1.0
1,b,0,3.0
2,b,1,1.0
3,b,1,3.0
4,a,2,0.0
5,a,2,2.0
6,c,3,NaN
7,a,4,0.0
8,a,4,2.0
9,b,5,1.0


In [39]:
df2 = pd.DataFrame({'key': ['a', 'b', 'a', 'b', 'd'],
                    'data2': range(5)})
df2

,key,data2
0,a,0
1,b,1
2,a,2
3,b,3
4,d,4


In [40]:
pd.merge(df1, df2, on='key', how='left')

,key,data1,data2
0,b,0,1.0
1,b,0,3.0
2,b,1,1.0
3,b,1,3.0
4,a,2,0.0
5,a,2,2.0
6,c,3,NaN
7,a,4,0.0
8,a,4,2.0
9,b,5,1.0


In [41]:
pd.merge(df1, df2, on='key', how='right')

,key,data1,data2
0,b,0.0,1
1,b,1.0,1
2,b,5.0,1
3,b,0.0,3
4,b,1.0,3
5,b,5.0,3
6,a,2.0,0
7,a,4.0,0
8,a,2.0,2
9,a,4.0,2


In [42]:
pd.merge(df1, df2, how='inner')

,key,data1,data2
0,b,0,1
1,b,0,3
2,b,1,1
3,b,1,3
4,b,5,1
5,b,5,3
6,a,2,0
7,a,2,2
8,a,4,0
9,a,4,2


In [43]:
left = pd.DataFrame({'key1': ['foo', 'foo', 'bar'],
                     'key2': ['one', 'two', 'one'],
                     'lval': [1, 2, 3]})
left

,key1,key2,lval
0,foo,one,1
1,foo,two,2
2,bar,one,3


In [44]:
right = pd.DataFrame({'key1': ['foo', 'foo', 'bar', 'bar'],
                      'key2': ['one', 'one', 'one', 'two'],
                      'rval': [4, 5, 6, 7]})
right

,key1,key2,rval
0,foo,one,4
1,foo,one,5
2,bar,one,6
3,bar,two,7


In [45]:
pd.merge(left, right, on=['key1', 'key2'], how='outer')

,key1,key2,lval,rval
0,foo,one,1.0,4.0
1,foo,one,1.0,5.0
2,foo,two,2.0,NaN
3,bar,one,3.0,6.0
4,bar,two,NaN,7.0


In [46]:
pd.merge(left, right, on='key1')

,key1,key2_x,lval,key2_y,rval
0,foo,one,1,one,4
1,foo,one,1,one,5
2,foo,two,2,one,4
3,foo,two,2,one,5
4,bar,one,3,one,6
5,bar,one,3,two,7


In [47]:
pd.merge(left, right, on='key1', suffixes=('_left', '_right'))

,key1,key2_left,lval,key2_right,rval
0,foo,one,1,one,4
1,foo,one,1,one,5
2,foo,two,2,one,4
3,foo,two,2,one,5
4,bar,one,3,one,6
5,bar,one,3,two,7


### merge函数的参数

left：合并时依据左边的，左边的关键值都保留

right：合并时依据右边的

how：inner，outer，left，right

on：需要连接的列名

left_on：左边表用作连接的列

right_on：右边表用作连接的列

left_index：使用左边的索引作为连接

right_index：使用右边索引进行连接

。。。

### 8.2.2 根据索引合并 Merging on Index

比如你有两个表，想合并

我们随便定义其中一个列表

In [ ]:
left1 = pd.DataFrame({'key': ['a', 'b', 'a', 'a', 'b', 'c'],
                      'value': range(6)})
left1

注意索引，a，b，c

定义第二个列表，其中之后两行

In [ ]:
right1 = pd.DataFrame({'group_val': [3.5, 7]}, index=['a', 'b'])
right1

使用merge合并两个列表

其中左边用left1，右边right1

左边的表用left_on='key'，key这一列

右边的表用index

In [ ]:
pd.merge(left1, right1, left_on='key', right_index=True)

这时，c不见了，只有a和b

顺序为先a，后b

how：inner，outer，left，right

how设置为outer

In [ ]:
pd.merge(left1, right1, left_on='key', right_index=True, how='outer')

这时c会保留，没有c的地方会记为NaN

### 多重索引的情况

多重索引的情况下，会更复杂

先定义一个放在左边的表

In [ ]:
lefth = pd.DataFrame({'key1': ['Ohio', 'Ohio', 'Ohio',
                               'Nevada', 'Nevada'],
                      'key2': [2000, 2001, 2002, 2001, 2002],
                      'data': np.arange(5.)})
lefth

定义右边的表，注意一下右边的index，就是我们要连接的依据

In [ ]:
righth = pd.DataFrame(np.arange(12).reshape((6, 2)),
                      index=[['Nevada', 'Nevada', 'Ohio', 'Ohio',
                              'Ohio', 'Ohio'],
                             [2001, 2000, 2000, 2000, 2001, 2002]],
                      columns=['event1', 'event2'])
righth

观察，索引有Navada,Ohio与年份两层

左边的设置为left_on=['key1', 'key2']

右边设置right_index=True，用索引链接

In [ ]:
pd.merge(lefth, righth, left_on=['key1', 'key2'], right_index=True)

默认情况下，两个表两层索引相同的列，数据会对齐

不相同的，删除

如果想要保留，则需要设置how='outer'

In [ ]:
pd.merge(lefth, righth, left_on=['key1', 'key2'],
         right_index=True, how='outer')

#### 使用两边的索引进行连接

定义左边的索引是ace

In [ ]:
left2 = pd.DataFrame([[1., 2.], [3., 4.], [5., 6.]],
                     index=['a', 'c', 'e'],
                     columns=['Ohio', 'Nevada'])
left2

右边，定义所以为bcde

In [ ]:
right2 = pd.DataFrame([[7., 8.], [9., 10.], [11., 12.], [13, 14]],
                      index=['b', 'c', 'd', 'e'],
                      columns=['Missouri', 'Alabama'])
right2

merge链接

设置参数

In [ ]:
pd.merge(left2, right2, how='outer', left_index=True, right_index=True)

#### join方法
join也可以实现按照索引合并

In [ ]:
left2.join(right2, how='outer')

on='key'的效果，添加一列叫做‘key’，以前的索引放到这一列


In [ ]:
left1.join(right1, on='key')

可以多个表链接在一起

这个跟之后要介绍的concat类似

In [ ]:
another = pd.DataFrame([[7., 8.], [9., 10.], [11., 12.], [16., 17.]],
                       index=['a', 'c', 'e', 'f'],
                       columns=['New York', 'Oregon'])
another

并排并到一起咯

In [ ]:
left2.join([right2, another])

设置how='outer'

In [ ]:
left2.join([right2, another], how='outer')

### 8.2.3 沿轴向连接Concatenating Along an Axis

concatenate

生成一个数组

In [ ]:
arr = np.arange(12).reshape((3, 4))
arr

两个表连起来axis=1

In [ ]:
np.concatenate([arr, arr], axis=1)

In [ ]:
np.concatenate([arr, arr], axis=0)

##### concat

三个索引不重叠的Series

In [ ]:
s1 = pd.Series([0, 1], index=['a', 'b'])
s2 = pd.Series([2, 3, 4], index=['c', 'd', 'e'])
s3 = pd.Series([5, 6], index=['f', 'g'])

默认axis=0

In [ ]:
pd.concat([s1, s2, s3])

如果设置axis=1

In [ ]:
pd.concat([s1, s2, s3], axis=1)

In [ ]:
s4 = pd.concat([s1, s3])
s4

In [ ]:
pd.concat([s1, s4], axis=1)

join='inner'，fg消失了

In [ ]:
pd.concat([s1, s4], axis=1, join='inner')

可以指定轴

In [ ]:
pd.concat([s1, s4], axis=1, join_axes=[['a', 'c', 'b', 'e']])

可以创建索引，区分拼接在一起的各部分

In [ ]:
result = pd.concat([s1, s1, s3], keys=['one', 'two', 'three'])
result
result.unstack()

In [ ]:
pd.concat([s1, s2, s3], axis=1, keys=['one', 'two', 'three'])

In [ ]:
df1 = pd.DataFrame(np.arange(6).reshape(3, 2), index=['a', 'b', 'c'],
                   columns=['one', 'two'])
df2 = pd.DataFrame(5 + np.arange(4).reshape(2, 2), index=['a', 'c'],
                   columns=['three', 'four'])

In [ ]:
df1

In [ ]:
df2

In [ ]:
pd.concat([df1, df2], axis=1, keys=['level1', 'level2'])

In [ ]:
pd.concat({'level1': df1, 'level2': df2}, axis=1)

In [ ]:
pd.concat([df1, df2], axis=1, keys=['level1', 'level2'],
          names=['upper', 'lower'])

In [ ]:
df1 = pd.DataFrame(np.random.randn(3, 4), columns=['a', 'b', 'c', 'd'])
df2 = pd.DataFrame(np.random.randn(2, 3), columns=['b', 'd', 'a'])
df1
df2

In [ ]:
pd.concat([df1, df2], ignore_index=True)

### 8.2.4 联合重叠数据 Combining Data with Overlap

In [ ]:
a = pd.Series([np.nan, 2.5, np.nan, 3.5, 4.5, np.nan],
              index=['f', 'e', 'd', 'c', 'b', 'a'])
b = pd.Series(np.arange(len(a), dtype=np.float64),
              index=['f', 'e', 'd', 'c', 'b', 'a'])
b[-1] = np.nan

In [ ]:
a

In [ ]:
b

In [ ]:
np.where(pd.isnull(a), b, a)

In [ ]:
b[:-2].combine_first(a[2:])

In [ ]:
df1 = pd.DataFrame({'a': [1., np.nan, 5., np.nan],
                    'b': [np.nan, 2., np.nan, 6.],
                    'c': range(2, 18, 4)})
df2 = pd.DataFrame({'a': [5., 4., np.nan, 3., 7.],
                    'b': [np.nan, 3., 4., 6., 8.]})
df1
df2
df1.combine_first(df2)

## 8.3 重塑和透视Reshaping and Pivoting

### 8.3.1 使用多层索引进行重塑 Reshaping with Hierarchical Indexing

In [ ]:
data = pd.DataFrame(np.arange(6).reshape((2, 3)),
                    index=pd.Index(['Ohio', 'Colorado'], name='state'),
                    columns=pd.Index(['one', 'two', 'three'],
                    name='number'))
data

In [ ]:
result = data.stack()
result

In [ ]:
result.unstack()

In [ ]:
result.unstack(0)

In [ ]:
result.unstack('state')

In [ ]:
s1 = pd.Series([0, 1, 2, 3], index=['a', 'b', 'c', 'd'])
s2 = pd.Series([4, 5, 6], index=['c', 'd', 'e'])
data2 = pd.concat([s1, s2], keys=['one', 'two'])
data2
data2.unstack()

In [ ]:
data2.unstack()
data2.unstack().stack()
data2.unstack().stack(dropna=False)

In [ ]:
df = pd.DataFrame({'left': result, 'right': result + 5},
                  columns=pd.Index(['left', 'right'], name='side'))
df
df.unstack('state')

In [ ]:
df.unstack('state').stack('side')

### 8.3.2 将长透视为宽 Pivoting “Long” to “Wide” Format

In [ ]:
data = pd.read_csv('examples/macrodata.csv')
data.head()
periods = pd.PeriodIndex(year=data.year, quarter=data.quarter,
                         name='date')
columns = pd.Index(['realgdp', 'infl', 'unemp'], name='item')
data = data.reindex(columns=columns)
data.index = periods.to_timestamp('D', 'end')
ldata = data.stack().reset_index().rename(columns={0: 'value'})

In [ ]:
ldata[:10]

In [ ]:
pivoted = ldata.pivot('date', 'item', 'value')
pivoted

In [ ]:
ldata['value2'] = np.random.randn(len(ldata))
ldata[:10]

In [ ]:
pivoted = ldata.pivot('date', 'item')
pivoted[:5]
pivoted['value'][:5]

In [ ]:
unstacked = ldata.set_index(['date', 'item']).unstack('item')
unstacked[:7]

### 8.3.3 将宽透视为长 Pivoting “Wide” to “Long” Format

In [ ]:
df = pd.DataFrame({'key': ['foo', 'bar', 'baz'],
                   'A': [1, 2, 3],
                   'B': [4, 5, 6],
                   'C': [7, 8, 9]})
df

In [ ]:
melted = pd.melt(df, ['key'])
melted

In [ ]:
reshaped = melted.pivot('key', 'variable', 'value')
reshaped

In [ ]:
reshaped.reset_index()

In [ ]:
pd.melt(df, id_vars=['key'], value_vars=['A', 'B'])

In [ ]:
pd.melt(df, value_vars=['A', 'B', 'C'])
pd.melt(df, value_vars=['key', 'A', 'B'])